# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import pandas as pd
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from sklearn.utils import resample
import joblib

In [2]:
ws = Workspace.from_config()
experiment_name = 'Wine_Evaluation_HyperDrive'

experiment = Experiment(ws, experiment_name)


## Dataset

The Wine Quality Datasets have been taken from the UCI Machine Learning Repository. The data is broken up into two individual datasets, one for red wines and the other for white wines. The red wine dataset contains 1599 examples while the white wine dataset has 4898 examples. Both datserts have the same 12 attributes as follows:

Attribute information:

For more information, read [Cortez et al., 2009].

Input variables (based on physicochemical tests):
1 - fixed acidity
2 - volatile acidity
3 - citric acid
4 - residual sugar
5 - chlorides
6 - free sulfur dioxide
7 - total sulfur dioxide
8 - density
9 - pH
10 - sulphates
11 - alcohol
Output variable (based on sensory data): 
12 - quality (score between 0 and 10)

There are no missing values. I will be combining these datasets into one dataset and using HyperDrive to tune a Logistic Regression model to predict if a wine is red or white.

This dataset is public and available for research. I have included the citation below as requested:

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

Available at: [@Elsevier] http://dx.doi.org/10.1016/j.dss.2009.05.016
            [Pre-press (pdf)] http://www3.dsi.uminho.pt/pcortez/winequality09.pdf
            [bib] http://www3.dsi.uminho.pt/pcortez/dss09.bib
            

            

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
dataset_name = 'winequality'

# Check Registry
if dataset_name in ws.datasets.keys(): 
    dataset = ws.datasets[dataset_name] 

    print(f'Located {dataset_name} in Registry.')

# Read from UCI ML Repository
else:
# Read data
    red = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
    white = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

# Set target classes (0: White Wine, 1: Red Wine)
    white['y'] = 0
    red['y'] = 1

# Combine into a single DataFrame & shuffle
    wine_df = pd.concat([red, white], axis=0)
    wine_df = wine_df.sample(frac=1)

# Upsample to balance dataset (Number of Instances: red wine - 1599; white wine - 4898)
    def upsample_classes(data, target):
        
        lst = list(data[target].unique())
        
        classes = []
        for c in lst:
            classes.append(data[data[target]==c])
        
        length = 0
        class_lab = None
        for c in classes:
            if len(c)>length:
                length=len(c)
                class_lab = c
        class_lab = class_lab[target].unique()[0]
        
        regroup = pd.concat(classes)
        maj_class = regroup[regroup[target]==class_lab]

        lst.remove(class_lab)
        
        new_classes=[]
        for i in lst:
            new_classes.append(resample(data[data[target]==i],replace=True, n_samples=len(maj_class)))

        minority_classes = pd.concat(new_classes)
        upsample = pd.concat([regroup[regroup[target]==class_lab],minority_classes])

        return upsample

    wine_df_balanced = upsample_classes(wine_df, 'y')


# Register the dataset
    datastore = ws.get_default_datastore()

    dataset = Dataset.Tabular.register_pandas_dataframe(
        dataframe=wine_df_balanced, 
        name=dataset_name, 
        description='A dataset of white and red wines',
        target=datastore
    )

    print(f'Read {dataset_name} from UCI ML Repository and registered dataset.')

Located winequality in Registry.


In [4]:
cluster_name = "hyperdrive-capstone-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

vm_size = "Standard_D2_V2"
max_nodes = 4

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, using it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

I used a Logistic Regression Classifier from Sklearn. My predictive task was binary classification and this model was 
appropriate. 

I chose to use tune 3 hyperparameters:
* *C*: Inverse of regularization strength; must be a positive float. Like in support vector machines, 
smaller values specify stronger regularization.
* *max_iter*: Maximum number of iterations taken for the solvers to converge.
* *solver*: Algorithm to use in the optimization problem.

Other possible hyperparameters were solver specific and were thus omitted. 
 
I used a RandomParameterSampling instead of an exhaustive GridSearch to reduce compute resources and time. This approach 
gives close to as good hyperparameters as a GridSearch with considerably less resources and time consumed. 

I used a BanditPolicy and set the evaluation_interval to 2 and the slack_factor to 0.1. This policy evaluates the primary 
metric every 2 iteration and if the value falls outside top 10% of the primary metric then the training process will stop. 
This saves time continuing to evaluate hyperparameters that don't show promise of improving our target metric. It prevents 
experiments from running for a long time and using up resources.

I chose the primary metric as Accuracy and to maximize it as part of the Hyperdrive run. I set the max total runs as 15 
to avoid log run times as well as the *max_concurrent_runs* to 4 as I am running this experiment on Standard_D2_V2 which 
has 4 nodes. This allows 4 jobs to be run in parallel on each node.


TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
    "C": choice(0.001, 0.01, 0.1, 1, 10),
    "max_iter": choice(25, 50, 100, 150, 200),
    "solver": choice('lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga')
    }
    )

#TODO: Create your estimator and hyperdrive config
if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', 
                script='train.py', 
                compute_target=cluster_name,
                environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
hyperparameter_sampling=param_sampling,
policy=early_termination_policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=15,
max_concurrent_runs=4
)

In [6]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_da1cd59b-2200-4b05-834e-022a01d359f9
Web View: https://ml.azure.com/runs/HD_da1cd59b-2200-4b05-834e-022a01d359f9?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-223774/workspaces/quick-starts-ws-223774&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-28T22:04:19.816949][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-28T22:04:20.8672963Z][SCHEDULER][INFO]Scheduling job, id='HD_da1cd59b-2200-4b05-834e-022a01d359f9_0' 
[2023-01-28T22:04:21.0267544Z][SCHEDULER][INFO]Scheduling job, id='HD_da1cd59b-2200-4b05-834e-022a01d359f9_1' 
[2023-01-28T22:04:21.1480313Z][SCHEDULER][INFO]Scheduling job, id='HD_da1cd59b-2200-4b05-834e-022a01d359f9_2' 
[2023-01-28T22:04:21.193108][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-01-28T22:04:21.5340339Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_da1cd59b-2200-4b05-8

{'runId': 'HD_da1cd59b-2200-4b05-834e-022a01d359f9',
 'target': 'hyperdrive-capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-01-28T22:04:19.015059Z',
 'endTimeUtc': '2023-01-28T22:13:30.223771Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '83f49237-fdb5-4e04-a709-44470d9e3919',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '125',
  'score': '0.9821428571428571',
  'best_child_run_id': 'HD_da1cd59b-2200-4b05-834e-022a01d359f9_5',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_da1cd59b-2200-4b05-834e-022a01d359f9_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Best Run
print("Best Run: ")
print("")

# Accuracy
print("Accuracy: " + str(best_run.get_metrics()['Accuracy']))
print("")

# Metrics
print("Metrics: " + str(best_run.get_metrics()))
print("")


# Tags
print("Tags: " + str(best_run.get_tags()))
print("")


Best Run: 

Accuracy: 0.9821428571428571

Metrics: {'Regularization Strength:': 1.0, 'Max iterations:': 50, 'Solver:': 'newton-cg', 'Accuracy': 0.9821428571428571}

Tags: {'_aml_system_hyperparameters': '{"C": 1, "max_iter": 50, "solver": "newton-cg"}', 'hyperparameters': '{"C": 1, "max_iter": 50, "solver": "newton-cg"}', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":4}'}



In [8]:
#TODO: Save the best model

# register model
best_run.register_model('hyperdrive-best-model.joblib', '.')

Model(workspace=Workspace.create(name='quick-starts-ws-223774', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-223774'), name=hyperdrive-best-model.joblib, id=hyperdrive-best-model.joblib:2, version=2, tags={}, properties={})

## Model Deployment

The AutoML model to predict wine type outperformed this Logistic Regression model trained and optimized using Hyperdrive. Thus there is no endpoint for this model and the AutoML model was deployed instead. The best accuracy from the AutoML model was 99.80% versus the best Hyperdrive model came with a best accuracy of 98.21%.




**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

